In [1]:
!pip install logzero
#Logging libraries
import logzero         
from logzero import logger
import pandas as pd
import re
import nltk
nltk.download('punkt')
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
#nltk.download('wordnet')

from nltk.tokenize import sent_tokenize, word_tokenize
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive


In [2]:
JobData = pd.read_pickle("/content/drive/My Drive/JobDescriptionPrediction/Resources/Data/interim/cleaned_indeed_job_dataset.pkl")
JobData.head(n=2)

,JobID,Job_Title,Queried_Salary,Job_Type,Skill,Company,Date_Since_Posted,Description,Location,Company_Industry
0,1,data scientist,<80000,data scientist,"sap, sql",Express Scripts,1.0,"[position summary, the business analyst rol...",MO,health care
1,2,data scientist,<80000,data scientist,"machine learning, r, sas, sql, python",Money Mart Financial Services,15.0,"[what do we need?, you to have an amazing p...",TX,other


In [3]:
JobData['Description'] = JobData['Description'].astype(str)   # Converting JobDescription to string

In [4]:
skill_synonyms = ['skill', 'skills', 'perform',  'accomplishment', 'qualification', 'qualifications', 'soft skills', 'experience', 'education', 'pedagogy']
role_synonyms = ['role', 'duty', 'responsibility', 'responsibilities','position description','what you will be doing', 'what you bring to the team']
whyus_synonyms = ['why','the team', 'the impact', 'company', 'what can you expect', 'business overview', 'about' ]


In [5]:
def segregate_description(input, index, include, exclude):
      
    reg_start = re.compile('('+'(.*)'+"|".join(include) + ')'+'(.*)')
  # skill_synonyms, role_synonyms,description_synonyms,experience_synonyms,education_synonyms
    reg_end = re.compile('('+'(.*)'+"|".join(exclude) +'(.*)'+ ')')
    start_flag=False

    output_skills = []
    for i in input:
      if(start_flag == False and re.match(reg_start,i)):
        start_flag=True
      if(start_flag == True and re.match(reg_end,i)):
        exit
      if(start_flag):
        output_skills.append(i)

      temp_df =pd.DataFrame()         
      temp_df['Output'] = output_skills
      temp_df['JobID'] = index
    #logger.info("-----Creating ", include, "Extracted Text-----------")
    return temp_df

def create_df(include, exclude):
  logger.info("-----Converting into dataframe-----------")
  df = pd.DataFrame(columns = ['JobID', 'Output'])
  for index, row in enumerate(JobData['Description']):
      input = sent_tokenize(JobData['Description'][index]) 
      output=segregate_description(input, index, include, exclude)
      df = df.append(output,ignore_index=True)
      
  return df


In [6]:
" Extracting skills"
skills_df = create_df(skill_synonyms,  whyus_synonyms+ role_synonyms)
JobData1 = JobData[['JobID', 'Job_Title', 'Skill']]
skills_final_df = pd.merge(JobData1, skills_df, how="inner", on=["JobID"])
skills_final_df = skills_final_df.drop_duplicates(keep= 'first')   # removing duplicates
skills_final_df= skills_final_df.groupby(skills_final_df['JobID'],as_index=False).agg({'Job_Title': 'first','Skill': 'first','Output':lambda x: x.tolist()   })
skills_final_df.to_excel("skills_dataset.xlsx", index=False)  
skills_final_df.to_pickle('skills_dataset.pkl')
logger.info("%sCreated Skills DataFrame  : ", skills_final_df.head(60))

[I 210411 05:57:53 <ipython-input-5-45beb8b2cbb7>:24] -----Converting into dataframe-----------
[I 210411 05:59:32 <ipython-input-6-b3410e0cb1ee>:9]     JobID  ...                                             Output
    0       1  ...  [any experience in statistical modeling fields...
    1       2  ...  [spss, sas, stata, r) required.experience with...
    2       3  ...  [ddi focuses on developing the workforce with ...
    3       4  ...  [[assist in consultations with business partne...
    4       5  ...  [[collecting and combining data from multiple ...
    5       7  ...  [, strong analytical skills and problem solvin...
    6       8  ...  [c c++, python, r, or similar languages).exper...
    7       9  ...  [qualifications   mandatory qualification crit...
    8      12  ...  [)., experience may not be substituted for the...
    9      13  ...  [, team working with:, currently 6 other data ...
    10     14  ...  [develops recommendations   formulates mathema...
    11     15  

In [7]:
" Extracting roles"
roles_df = create_df(role_synonyms,  whyus_synonyms+skill_synonyms )
JobData2 = JobData[['JobID', 'Job_Title', 'Company_Industry']]
roles_final_df = pd.merge(JobData2, roles_df, how="inner", on=["JobID"])
roles_final_df = roles_final_df.drop_duplicates(keep= 'first')   # removing duplicates
roles_final_df= roles_final_df.groupby(roles_final_df['JobID'],as_index=False).agg({'Job_Title': 'first', 'Company_Industry': 'first','Output':lambda x: x.tolist()   })
roles_final_df.to_excel("roles_dataset.xlsx", index=False)  
roles_final_df.to_pickle('roles_dataset.pkl')
logger.info("%sCreated Roles DataFrame  : ", skills_final_df.head(60))

[I 210411 05:59:32 <ipython-input-5-45beb8b2cbb7>:24] -----Converting into dataframe-----------
[I 210411 06:01:01 <ipython-input-7-2cf803572ccb>:9]     JobID  ...                                             Output
    0       1  ...  [any experience in statistical modeling fields...
    1       2  ...  [spss, sas, stata, r) required.experience with...
    2       3  ...  [ddi focuses on developing the workforce with ...
    3       4  ...  [[assist in consultations with business partne...
    4       5  ...  [[collecting and combining data from multiple ...
    5       7  ...  [, strong analytical skills and problem solvin...
    6       8  ...  [c c++, python, r, or similar languages).exper...
    7       9  ...  [qualifications   mandatory qualification crit...
    8      12  ...  [)., experience may not be substituted for the...
    9      13  ...  [, team working with:, currently 6 other data ...
    10     14  ...  [develops recommendations   formulates mathema...
    11     15  

In [8]:
" Extracting WhyUS"
whyus_df = create_df(whyus_synonyms,  role_synonyms+skill_synonyms )
JobData3 = JobData[['JobID','Company', 'Location', 'Company_Industry']]
whyus_final_df = pd.merge(JobData3, whyus_df, how="inner", on=["JobID"])
whyus_final_df = whyus_final_df.drop_duplicates(keep= 'first')   # removing duplicates
whyus_final_df= whyus_final_df.groupby(whyus_final_df['JobID'],as_index=False).agg({'Company': 'first', 'Location': 'first','Output':lambda x: x.tolist()   })
whyus_final_df.to_excel("whyus_dataset.xlsx", index=False)
whyus_final_df.to_pickle('whyus_dataset.pkl')  
logger.info("%sCreated Why Us DataFrame  : ", whyus_final_df.head(10))

[I 210411 06:01:02 <ipython-input-5-45beb8b2cbb7>:24] -----Converting into dataframe-----------
[I 210411 06:02:20 <ipython-input-8-08811d98647c>:9]    JobID  ...                                             Output
    0      1  ...  [and most importantly that you have unquestion...
    1     13  ...  [[mid data scientist, our client in the midtow...
    2     14  ...  [[why catalina   , catalina’s personalized dig...
    3     27  ...  [2. in the “cover letter other notes section:”...
    4     28  ...  [2. in the “cover letter other notes section:”...
    5     32  ...  [2. in the “cover letter other notes section:”...
    6     34  ...  [2. in the “cover letter other notes section:”...
    7     40  ...  [capable problem solver who uses logic to crea...
    8     42  ...  [, a few reasons why this is the #bestjobever,...
    9     53  ...  [that’s why we’re committed to making florida ...
    
    [10 rows x 4 columns]Created Why Us DataFrame  : 


In [9]:
logger.info("%s Segregated Job Descriptipn ")       
logzero.logfile("logdatasegregation.log",maxBytes=1e6)

[I 210411 06:02:20 <ipython-input-9-f27f7148621d>:1] %s Segregated Job Descriptipn 


In [10]:
print(logger)

<Logger logzero_default (DEBUG)>
